## Redis Stats Guidance

In this notebook, we will explain what is stored and provided inside the redis database. What is the structure of each type of data including agent profile, relationship profile, environment profile, agentenv combo profile, and episodelog profile. Additionally, this notebook provides statistical information about how many datapoints are inside the provided database.

In [6]:
import os
import sys
import rich
from collections import Counter
from sotopia.database.persistent_profile import (
    AgentProfile,
    EnvironmentProfile,
    RelationshipProfile,
)
from sotopia.database.logs import EpisodeLog
from sotopia.database.env_agent_combo_storage import EnvAgentComboStorage

sys.path.append("../")
# os.environ["REDIS_OM_URL"] = "redis://:QzmCUD3C3RdsR@localhost:6379"
os.environ["REDIS_OM_URL"]="redis://:@localhost:6379"


## Relationship Profile

RelationshipProfile stores the information about different relationship between agents.

In [5]:
res_pks = RelationshipProfile.all_pks()
res_pks = list(res_pks)
print(len(res_pks))
res = []
for pk in res_pks:
    try:
        res.append(RelationshipProfile.get(pk=pk))
    except Exception:
        print("error")
        pass
res_relationships = [r.relationship for r in res]
Counter(res_relationships)

TimeoutError: Timeout connecting to server

## Agents Profile

AgentProfile stores the information about each agent.

In [ ]:
# obtain a specific agent
agents = AgentProfile.find(AgentProfile.first_name == "ss").all()
rich.print(agents)

In [ ]:
# find specific agnets
agents = AgentProfile.find(AgentProfile.gender == "Man", AgentProfile.age > 30)
agents = agents.all()
print(len(agents))
rich.print(agents[0])

In [ ]:
# obtain all agents' basic info
agent_pks = AgentProfile.all_pks()
agent_pks = list(agent_pks)
agents = []
for pk in agent_pks:
    try:
        agents.append(AgentProfile.get(pk=pk))
    except Exception:
        print("error")
        pass
print(len(agents))
rich.print(agents[0])

In [ ]:
agent_pks = AgentProfile.all_pks()
agent_pks = list(agent_pks)
print(len(agent_pks))

In [ ]:
# Update agent's information
agents = AgentProfile.find(
    AgentProfile.first_name == "Ava", AgentProfile.last_name == "Martinez"
).all()[0]

In [ ]:
agents.update(secret="Keeps their bisexuality a secret from her conservative family")

## Environment Profile

EnvironmentProfile stores the information about social scenario. 

In [ ]:
# get all environments
all_envs = list(EnvironmentProfile.all_pks())
print(len(all_envs))
print(all_envs[:5])

In [ ]:
# get a specific environment profile
env_profile_id = all_envs[0]
env = EnvironmentProfile.get(env_profile_id)
rich.print(env)

## EnvAgentComboStorage
Identify the combination of environment and agent that is used in the episodes.
Once we have the combination, we can use it to start the simulation.
Combo is a combination of Environment and two agents.

In [ ]:
# all env-agent combos
all_combos = EnvAgentComboStorage().all_pks()
all_combos = list(all_combos)
print(len(all_combos))
rich.print(EnvAgentComboStorage().get(all_combos[0]))

## EnvironmentList
Store a list of special environments (e.g., sotopia hard) that can be used to start certain simulations. Agent index is used to identify the special agent in the simulation.

In [ ]:
from sotopia.database.persistent_profile import EnvironmentList

all_list = EnvironmentList.all_pks()
all_list = list(all_list)
print(len(all_list))

In [ ]:
from sotopia.samplers import ConstraintBasedSampler
from sotopia.messages import AgentAction, Observation
from sotopia.agents import LLMAgent
import json
# In this example, we will demonstrate using the EnvironmentList class to sample a list of EnvAgentComboStorage and serialize it to a json file that can be used for sharing with others for benchmarking purposes.


def _sample_env_agent_combo_and_push_to_db(env_id: str) -> list[EnvAgentComboStorage]:
    combo_list = []
    sampler = ConstraintBasedSampler[Observation, AgentAction](env_candidates=[env_id])
    env_agent_combo_list = list(
        sampler.sample(agent_classes=[LLMAgent] * 2, replacement=False, size=10)
    )
    for env, agent in env_agent_combo_list:
        combo = EnvAgentComboStorage(
            env_id=env.profile.pk,
            agent_ids=[agent[0].profile.pk, agent[1].profile.pk],
        )
        combo_list.append(combo)
    return combo_list


# First we will extrat the hard environments from the EnvironmentList
hard_envs = EnvironmentList.get("01HAK34YPB1H1RWXQDASDKHSNS").environments
print(len(hard_envs))
hard_envs_set = set(hard_envs)

# Next we will sample 10 EnvAgentComboStorage from each hard environment
final_list_for_benchmark_agents = []
for env in hard_envs_set:
    combo_list = EnvAgentComboStorage.find(EnvAgentComboStorage.env_id == env).all()
    print(len(combo_list))
    final_list_for_benchmark_agents.extend(combo_list)

# Finally we will serialize the list to a json file
with open("../data/benchmark_agents.json", "w") as f:
    json.dump(
        [combo.dict() for combo in final_list_for_benchmark_agents],
        f,
        indent=4,
    )

In [ ]:
EnvironmentList.get("01HAK34YPB1H1RWXQDASDKHSNS")

## Episode Log

In [ ]:
# find episode log by tag
Episodes = EpisodeLog.find(EpisodeLog.tag == "aug20_gpt4_llama-2-70b-chat_zqi2").all()
len(Episodes)  ## Episode Log

## Episodelog stores the social conversation between two agents in an environment.

In [ ]:
# get all episode logs' primary keys
episode_pks = EpisodeLog.all_pks()
episode_pks = list(episode_pks)
print(len(episode_pks))
print(episode_pks[0])

In [ ]:
test_ep = EpisodeLog.get(episode_pks[0])
agent_profiles, conversation = test_ep.render_for_humans()
for agent_profile in agent_profiles:
    rich.print(agent_profile)
for message in conversation:
    rich.print(message)

In [ ]:
# get the epilogs that contain the specified models
model1 = "gpt-4"
model2 = "gpt-4"
model_comp1 = ["gpt-4", model1, model2]
model_comp2 = ["gpt-4", model2, model1]

gpt4_gpt4_eps = []
for epid in episode_pks:
    try:
        curr_ep = EpisodeLog.get(epid)
    except Exception:
        continue
    if curr_ep.models == model_comp1 or curr_ep.models == model_comp2:
        gpt4_gpt4_eps.append(curr_ep)
len(gpt4_gpt4_eps)

In [ ]:
agent_profiles, conversation = gpt4_gpt4_eps[0].render_for_humans()
for agent_profile in agent_profiles:
    rich.print(agent_profile)
for message in conversation:
    rich.print(message)